Tech Stack

*   Python using spotify and gspread
*   Jupyter notebook
*   API keys from spotify and google for accessing user data
*   Glide for app creation

In [6]:
#install libraries
pip install spotipy
pip install gspread

SyntaxError: invalid syntax (<ipython-input-6-122b8b35515e>, line 2)

In [1]:
#import libraries
import requests # learn 
import spotipy
from spotipy.oauth2 import SpotifyOAuth  # learn 
import pandas as pd
import gspread

In [2]:
# set-up variables based on app in spotify developers dashboard
# redirect can be any URI
# scope is required for reading the top songs/artists
SPOTIPY_CLIENT_ID='8a1696ba9459480d9aa8c9f8109f2f41'
SPOTIPY_CLIENT_SECRET='88475b94f0a24cd7b752ab558288ea9a'
SPOTIPY_REDIRECT_URI='http://127.0.0.1:9090'
SCOPE = "user-top-read"

In [3]:
# establish authentification
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET, redirect_uri=SPOTIPY_REDIRECT_URI, scope=SCOPE))

In [4]:
# get the top tracks
results = sp.current_user_top_tracks()

Use parameters to filter results
               
                - limit - the number of entities to return
                - offset - the index of the first entity to return
                - time_range - Over what time frame are the affinities computed

In [5]:
# get a shortened list of top tracks
top_tracks_short = sp.current_user_top_tracks(limit=10, offset=0, time_range='short_term')
top_tracks_long = sp.current_user_top_tracks(limit=10, offset=0, time_range='long_term')

In [6]:
# use the top tracks and get the track ids for each
def get_track_ids(time_frame):
 track_ids = []
 for song in time_frame['items']:
  track_ids.append(song['id'])
 return track_ids

In [7]:
track_ids = get_track_ids(top_tracks_short)
track_ids

['41HyMiZSBLGlfopnDE7Ppc',
 '5tKD7VkJxsEWBx5sKCsZTo',
 '27ShY3jgbdeXwbMHmpDqPw',
 '3iDHkSTaDcFzsqGCsv5S5J',
 '1HVYQj4liQb1QOygBfjZqV',
 '45ZOQlyz2XJpQREyKWzhe7',
 '7cp9Cs1rEtNQqKuawmuiJp',
 '1Q0fgWmTiGf9yixErBPgFi',
 '44KSiIcCon2uw4pDWjZJww',
 '4TvxDMTOkbK2yt7IfHWqZQ']

In [8]:
## FOR BELOW LOOK AT HOW META LOOKS

In [9]:
# get different features from the track
def get_track_features(id):
 meta = sp.track(id)
 # meta
 name = meta['name']
 album = meta['album']['name']
 artist = meta['album']['artists'][0]['name']
 spotify_url = meta['external_urls']['spotify']
 album_cover = meta['album']['images'][0]['url']
 track_info = [name, album, artist, spotify_url, album_cover]
 return track_info

In [24]:
# insert time
import time
from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d')

In [ ]:
# save each time periods tracks to a different csv

In [25]:
def convert_to_df(track_ids):
 # loop over track ids 
    tracks = []
    for i in range(len(track_ids)):
        time.sleep(.5)
        track = get_track_features(track_ids[i])
        tracks.append(track)
 # create dataset
    df = pd.DataFrame(tracks, columns = ['name', 'album', 'artist', 'spotify_url', 'album_cover'])
 # save to CSV
    df.to_csv(f'{today}-{time_period}.csv')

In [26]:
time_ranges = ['short_term', 'medium_term', 'long_term']
for time_period in time_ranges:
    top_tracks = sp.current_user_top_tracks(limit=20, offset=0, time_range=time_period)
    track_ids = get_track_ids(top_tracks)
    convert_to_df(track_ids)

short_term
medium_term
long_term


In [ ]:
# send to a google sheet file

In [39]:
# connect to 
gc = gspread.service_account(filename='spotifywrapped-6f202e61259c.json')

In [42]:
sh = gc.open('My Spotify Wrapped')
worksheet = sh.worksheet('short_term')
val = worksheet.acell('A5').value
print(val)

Head on Fire - Danny L Harle Remix


In [45]:
worksheet.update([df.columns.values.tolist()] + df.values.tolist())

{'spreadsheetId': '1wN9ql9byD6c9ES2J4-lf8VOMLNoC6eYah11Q0VONg4M',
 'updatedRange': 'short_term!A1:E11',
 'updatedRows': 11,
 'updatedColumns': 5,
 'updatedCells': 55}

In [48]:
# insert into sheet
def insert_to_gsheet(track_ids):
    # loop over track ids 
    tracks = []
    for i in range(len(track_ids)):
        time.sleep(.5)
        track = get_track_features(track_ids[i])
        tracks.append(track)
    # create dataset
    df = pd.DataFrame(tracks, columns = ['name', 'album', 'artist', 'spotify_url', 'album_cover'])
    # insert into google sheet
    gc = gspread.service_account(filename='spotifywrapped-6f202e61259c.json')
    sh = gc.open('My Spotify Wrapped')
    worksheet = sh.worksheet(f'{time_period}')
    worksheet.update([df.columns.values.tolist()] + df.values.tolist())
    print('Done')

In [50]:
time_ranges = ['short_term', 'medium_term', 'long_term']
for time_period in time_ranges:
    top_tracks = sp.current_user_top_tracks(limit=20, offset=0, time_range=time_period)
    track_ids = get_track_ids(top_tracks)
    insert_to_gsheet(track_ids)

Done
Done
Done


to do:

*   gui
*   make executable
*   test with multiple users


